In [6]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply
from cognite.client.exceptions import CogniteException

from tests.auth_utils import get_toolkit_client

In [7]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [8]:
SPACE = "sp_upgrade"

In [9]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Delete Asset-Centric FileMetadata

In [10]:
file = FileMetadataWrite(
    external_id="ts_case4_delete_asset_centric",
    name="Case 4 Cascading Delete",
)

In [11]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case4_delete_asset_centric
name,Case 4 Cascading Delete
labels,[]
id,7849972293300329
created_time,2025-07-04 10:03:24.741000
last_updated_time,2025-07-04 10:03:24.741000
uploaded,False


In [12]:
filecontent = b"This is filecontent"

In [13]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,ts_case4_delete_asset_centric
name,Case 4 Cascading Delete
id,7849972293300329
created_time,2025-07-04 10:03:24.741000
last_updated_time,2025-07-04 10:03:24.741000
uploaded,False


In [ ]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    updated

In [ ]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name=file.name,
)

In [ ]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

In [ ]:
retrieved_ts = client.files.retrieve(external_id=file.external_id)
retrieved_ts

In [ ]:
try:
    client.time_series.delete(external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    print(f"Deleted Time Series with external_id: {file.external_id}")